#  Практика по работе с Энкодерами

В этом задании мы потренируемся использовать различные энкодеры под наши задачи. Мы поработаем с классической бинарной классификацией отзывов на фильмы, попробуем усложнить и выйти на многоклассовую задачу, а после - подключим определение семантической близости фильмов по их описанию.

### Бинарная классификация

In [2]:
!pip install torch --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 189.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

In [1]:
import datasets
import fsspec
import torch
print(datasets.__version__)  # Должно быть 3.6.0
print(fsspec.__version__)   # Должно быть 2025.3.2
print(torch.__version__)    # Проверь версию torch

3.6.0
2025.3.2
2.6.0+cu124


In [2]:
import numpy as np
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)

SEED = 42
set_seed(SEED)

Перед вами практическое задание по бинарной классификации текстов с использованием предобученной модели BERT. Мы будем решать задачу определения тональности пользовательских отзывов на фильмы — **положительный** отзыв или **отрицательный**.

В качестве источника данных используем открытый датасет [IMDb](https://huggingface.co/datasets/stanfordnlp/imdb) от [Stanford NLP](https://nlp.stanford.edu/), содержащий 50 000 англоязычных рецензий, размеченных вручную. Цель — построить и обучить модель, способную автоматически классифицировать тексты по их эмоциональной окраске. Вы пройдёте все основные этапы пайплайна: от очистки текста и токенизации до обучения модели и оценки качества.

Перед тем, как мы двинемся дальше, проверьте включен или выключен GPU в colab. Работа с BERT потребует ресурсов, но предлагаем сначала заполнить весь необходимый код, а потом уже подключить GPU перед самым обучением модели. Советуем использовать доступную в colab Tesla T4.

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU на месте: {torch.cuda.get_device_name(0)}")

In [4]:
# датасет сразу подгружен в модуль datasets от HF

dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Мы видим, что в датасете уже подготовленное разделение данных на тренировочную и тестовую выборку, а также "unsupervised" набор - неразмеченная часть данных — то есть отзывы, у которых нет корректной метки (label не несёт обучающей информации).

* В части `train` и `test` у каждого отзыва есть метка: `label = 0` (негативный) или `label = 1` (позитивный).
* В части `unsupervised` колонка `label` есть, но её значение не используется — оно либо пустое, либо фиктивное (`-1`), так как эта часть предназначена для:

  * обучения без учителя (например, pretraining),
  * самостоятельного дообучения языковой модели,
  * полу- или слаборазмеченного обучения.

In [22]:
dataset["unsupervised"][0] # лейбл обозначен как "-1"

{'text': 'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)',
 'label': -1}

В нашей задаче мы не будем использовать `unsupervised` часть — только размеченные данные из `train` и `test`. Точнее, только из `train` - 50.000 объектов на обучение это неплохо, но для учебной задачи долговато. С целью сокращения времени - сократим и датасет.

**Задание 1.** Давайте оставим только `train`, а затем перемешаем его с помщью `shuffle`, а после разобьем на обучающую и валидационную выборки в соотношении 80:20.

**Задание курса**

> Попробуем удержать баланс классов? Установите в гиперпараметре пропорциональное распределение классов. Если не помните, как это выполнить, загляните в описание метода train_test_split.

In [30]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Загружаем датасет
dataset = load_dataset("stanfordnlp/imdb", split="train")
df = dataset.to_pandas()

# Определяем признаки и метки
X = df['text']  # Признаки: текст отзывов
y = df['label']  # Целевая метка: 0 (негативный) или 1 (позитивный)

# Разделяем на обучающую и валидационную выборки
train_data, val_data, train_labels, val_labels = train_test_split(
    X, y, test_size=0.2, train_size=0.8, random_state=42, shuffle=True, stratify=y
)

# Проверяем
print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Train labels distribution:", train_labels.value_counts())
print("Validation labels distribution:", val_labels.value_counts())

Train data shape: (20000,)
Validation data shape: (5000,)
Train labels distribution: label
1    10000
0    10000
Name: count, dtype: int64
Validation labels distribution: label
1    2500
0    2500
Name: count, dtype: int64


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(train_data)
X_val_tfidf = vectorizer.transform(val_data)

print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_val_tfidf shape:", X_val_tfidf.shape)

X_train_tfidf shape: (20000, 5000)
X_val_tfidf shape: (5000, 5000)


In [25]:
print(vectorizer.get_feature_names_out()[:10])

['00' '000' '10' '100' '11' '12' '13' '13th' '14' '15']


применяем алгоритм классификации LogisticRegression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000) # количество итераций для оптимизации алгоритма (по умолчанию 100)
model.fit(X_train_tfidf, train_labels)
predictions = model.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(val_labels, predictions))

Accuracy: 0.882


Проведение эксперимента:
1. поменяем количество итераций на 100, 500 и проверим точность
2. пробуем другой solver-оптимизатор

In [28]:
# 100
model = LogisticRegression(max_iter=100)
model.fit(X_train_tfidf, train_labels)
predictions = model.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(val_labels, predictions))

# 500
model = LogisticRegression(max_iter=500)
model.fit(X_train_tfidf, train_labels)
predictions = model.predict(X_val_tfidf)
print("Accuracy:", accuracy_score(val_labels, predictions))

# solver = 'saga'
model = LogisticRegression(max_iter=1000, solver='saga')
model.fit(X_train_tfidf, train_labels)
predictions = model.predict(X_val_tfidf)
print("Accuracy with solver='saga':", accuracy_score(val_labels, predictions))

Accuracy: 0.882
Accuracy: 0.882
Accuracy with solver='saga': 0.8818


меняем алгоритм на MultinomialNB и замеряем точность

In [29]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_tfidf, train_labels)
predictions = model.predict(X_val_tfidf)
print("Naive Bayes Accuracy:", accuracy_score(val_labels, predictions))

Naive Bayes Accuracy: 0.8502


**Вопрос 1.** Сколько отзывов положительного класса `(label = 1)` содержится в `val_df` после разбиения?

> 2500

>

**Задание 2.** **А стоит ли чистить отзывы?**

Перед тем как обучать модель на текстовых данных, важно понять, в каком виде приходят тексты, и нужна ли им предобработка.

> Добавить блок с цитатой



В нашем случае отзывы взяты напрямую из `IMDb` — они написаны живыми пользователями, без фильтрации и нормализации. Чтобы принять решение об очистке, посмотрите на первые несколько примеров из датасета.

Обратите внимание на:

* наличие заглавных букв,
* пунктуацию (много ли её? нарушает ли она понимание?),
* цифры и HTML-теги (встречаются ли? нужны ли они для тональности?),
* стоп-слова (могут ли они мешать?).

In [ ]:
# посмотрите на первые десять примеров из датасета (можно взять и больше) и поизучайте их



**Вопрос 2** - на размышление: *Какую базовую предобработку вы бы применили к этим отзывам перед подачей в токенизатор BERT?*

> *Важно помнить*: модели `BERT` обучаются на сырых текстах, но легкая очистка всё же может помочь — особенно, если мы визуализируем текст, считаем TF-IDF или хотим сделать простой анализ.

В следующем шаге вы напишете функцию очистки текста. Если вы считаете, что некоторые из этих элементов обработки не нужны, всё равно, пожалуйста, выполните их. В бонусном задании вы сможете вернуться к этому вопросу и поэкспериментировать с разными подходами к обработке.

**Задание 3.** Теперь почистим наши тексты. Напишите функцию `clean_text`, которая будет очищать текст от лишнего. Вы можете опираться на код с семинара или написать свой вариант. Давайте: 1) приведем к нижнему регистру 2) уберем числа 3) удалим переносы строк 4) уберем стоп-слова.

Также удалите из таблицы те строки, где текст отзыва оказался пустым после очистки.

Вам здесь пригодятся модули `re`, `string`, `nltk` (`nltk.download('stopwords')`), `stopwords` из `nltk.corpus`. Не забудьте применить обработку и к трейну, и к тесту.

In [ ]:
# здесь ваш код
# (⌒_⌒;)

**Вопрос 3.** Чему равна средняя длина очищенных отзывов (в символах) в `train_df`?

**Задание 4.** Теперь реализуем функцию оценки качества. При обучении модели через `Trainer` из библиотеки 🤗 можно передать свою функцию подсчёта метрик. Это позволяет отслеживать не только `loss`, но и, например, F1-score, accuracy или другие метрики качества.


Мы хотим реализовать функцию `compute_metrics`, которая будет передаваться в `Trainer`. Эта функция получает на вход кортеж `(logits, labels)` — предсказания модели и реальные метки, и должна возвращать **accuracy** (долю правильных ответов).

**Вопрос 4.** Почему, кстати, мы выбрали здесь именно **accuracy** метрику?



In [ ]:
# здесь импуты, если потребуются

def compute_metrics(eval_pred):

    # вход: пара (logits, labels)
    # верните словарь с точностью

    return

Запустите следующий код. Если ваша функция написана верно, то код отработает без ошибок.

In [ ]:
test_logits = np.array([[0.1, 0.9], [0.8, 0.2], [0.4, 0.6]])
test_labels = np.array([1, 0, 1])

eval_pred = (test_logits, test_labels)

metrics = compute_metrics(eval_pred)
print("Test metrics:", metrics)

assert "accuracy" in metrics, "Функция должна возвращать словарь с ключом 'accuracy'"
assert isinstance(metrics["accuracy"], float), "Значение accuracy должно быть числом"
assert abs(metrics["accuracy"] - 1.0) < 1e-6, "Ожидаемое значение accuracy: 1.0 на идеально предсказанном примере"

Test metrics: {'accuracy': 1.0}


**Задание 5.** Реализуем функцию токенизации текста. Перед тем как подавать тексты в модель `BERT`, нужно преобразовать их в числовой формат — токены. Это делает токенизатор модели.

Токенизатор превращает каждый отзыв в набор чисел, соответствующих подсловным единицам, которые `BERT` использует как вход. Но важно задать параметры, чтобы все примеры имели одинаковую длину и не обрезались неконтролируемо.


Реализуйте функцию `tokenize_function`, которая применяет токенизатор к колонке с очищенными отзывами. Мы будем использовать *построчную токенизацию с паддингом и усечением*.


In [ ]:
def tokenize_function(examples):
    return # здесь ваш код

**Вопрос 5.** Какой набор параметров токенизатора будет наиболее подходящим для обучения модели классификации отзывов?

Теперь давайте немного порассуждаем насчет гиперпараметра `max_length`: какое количество токенов мы будет отправлять в нашу модель? Итак, что мы уже знаем:
1. максимум для `BERT` это `max_length = 512`.
2. В английском языке при использовании токенизации WordPiece (у `bert-base-uncased`, например) из 100 символов в среднем получается от 20 до 40 токенов, в зависимости от слов, пунктуации и наличия редких подслов.



In [ ]:
# давайте проверим

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = df["clean_text"].iloc[0]
tokens = tokenizer.tokenize(text)

print(len(text), "символов")
print(len(tokens), "токенов")

# 1127 символов
# 223 токенов

Если средняя длина отзывов после очистки — около 900 символов, то разумный выбор:

* `max_length = 256` — безопасный и быстрый вариант
* `max_length = 384` — если хочется сохранить больше контекста
* `max_length = 512` — максимум для **BERT**, но:

  * увеличивает время и потребление памяти
  * может быть избыточен (для коротких отзывов)



**Задание 6.** Подобрались мы и к обучению. Теперь, когда вы освоили все этапы предобработки, токенизации и настройки модели, давайте применим эти знания на практике и сравним три разных версии BERT на задаче классификации отзывов.



Это будет самая сложная часть - но и самая содержательная. Цель задания:

1. Написать **полный пайплайн обучения модели**, включая токенизацию (подтягиваем нашу функцию, написанную выше), обучение, валидацию.
2. Запустить пайнлайн с тремя предобученными моделями:
   * `"bert-base-uncased"`
   * `"roberta-base"`
   * `"distilbert-base-uncased"`
3. Обучить каждую модель на 3 эпохах и сравнить метрику качества (`accuracy`).

PS Вы можете написать логику для каждой модели отдельно, но будет более изящно, если вы объедините всё в единый цикл - без лишнего дублирования кода.

> Почитайте, что такое [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorWithPadding:~:text=value%20at%20initialization.-,DataCollatorWithPadding,-class%20transformers.) и попробуйте интегрировать в своё решение.

In [ ]:
# Аргументы обучения - пусть будут едины, чтобы мы согли синхронизировать результаты
args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    report_to="none" # отключаем wandb и tensorboard - вы можете подключить, если это нужно
)

In [ ]:
# здесь ваш код
# (⌒_⌒;)

**Бонусное задание.** Выберете модель: можно взять из числа предложенных или любую другую - и попробуйте максимально увеличить метрики на валидации. Можно экспериментировать с методами обработки текстов, с архитектурами модели, подбором гиперпараметров и тд. Можно и учеличить объем данных - если вам позволит время и ресурсы.

Напишите о своих успехах нам на степике в комментариях под бонусным вопросом. И студентам, и нам будет очень интересно прочесть ваши эксперименты и результаты.

In [ ]:
# здесь ваш код
# ヽ(♡‿♡)ノ

In [ ]:
# Посмотрим на первые 10 отзывов из train_df
if 'train_df' in locals() and not train_df.empty:
    print("Первые 10 отзывов:")
    for i in range(min(10, len(train_df))):
        print(f"--- Отзыв {i+1} ---")
        print(train_df['text'].iloc[i])
        print("-" * 20)
else:
    print("DataFrame 'train_df' не найден или пуст. Пожалуйста, убедитесь, что данные были загружены и разбиты успешно.")